# High-Level API

These are the main objects used in the high-level API to create models, intelligently feed samples into them, and experiment with hyperparameters.

| High-level object |                    Groups together the following objects                   |
|:-----------------:|:--------------------------------------------------------------------------:|
|    `Algorithm`    | Functions to build, train, predict, and evaluate a machine learning model. |
|   `DataPipeline`  |      Dataset, Label, Featureset, Splitset, Foldset, Folds, Preprocess.     |
|    `Experiment`   |    Algorithm, Hyperparamset, Hyperparamcombos, DataPipeline, Batch, Job.   |

In this way, the high-level API abstracts the lower-level API to make things easier for the user.

## Prerequisites
If you've already completed the instructions on the **Installation** page, then let's get started.

In [1]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Usage

### 1. Algorithm

An `Algorithm` is the ORM's codename for a machine learning model since *Model* is the most important *reserved word* for ORMs.

In [2]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

You can name the functions below whatever you want, but do not change the predetermined `*args` (e.g. `**hyperparameters`, `model`, etc.).

Put a placeholder anywhere you want to try out different hyperparameters: `hyperparameters['<some_variable_name>']`. You'll get a chance to define the hyperparameters in a minute.

In [3]:
def function_model_build(**hyperparameters):
    model = Sequential()
    model.add(Dense(13, input_shape=(4,), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(hyperparameters['l2_neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(3, activation='softmax', name='output'))

    model.compile(optimizer=hyperparameters['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [4]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hyperparameters['epochs']
        , callbacks=[History()]
    )
    return model

Then pass these functions into the `Algorithm`.

The `library` and `analysis_type` help handle the model and its output behind the scenes. Current analysis types include: 'classification_multi', 'classification_binary', and 'regression'.

In [5]:
algorithm = aiqc.Algorithm.make(
	library = "keras"
	, analysis_type = "classification_multi"
	, function_model_build = function_model_build
	, function_model_train = function_model_train
)

#### Hyperparameters

The `hyperparameters` below will be automatically fed into the functions above as `**kwargs` via the `**hyperparameters` argument we saw earlier.

For example, wherever you see `hyperparameters['neuron_count']`, it will pull from the *key:value* pair `"neuron_count": [9, 12]` seen below. Where model A will have 9 neurons and model B will have 12 neurons.

In [6]:
hyperparameters = {
	"neuron_count": [9, 12]
	, "learning_rate": [0.03, 0.05]
    , "epoch_count": [25, 50]
}

### 2. DataPipeline

In [7]:
from sklearn.preprocessing import *

In [8]:
file_path = examples.demo_file_to_pandas('iris.tsv')

In [9]:
datapipeline = aiqc.DataPipeline.make(
	dataFrame_or_filePath = file_path
	, label_column = 'species'
	, size_test = 0.24
	, size_validation = 0.16
	, fold_count = None
	, encoder_features = StandardScaler()
	, encoder_labels = OneHotEncoder(sparse=False)
)

> Don't use `fold_count` unless your (total sample count / fold_count) still gives you an accurate representation of your sample population. You can try it with the 'iris_10x.tsv' demo_file.
>
> In the future, we'll turn the `encoder_*` attributes into customizable functions.

### 3. Experiment

Now it's time to bring together the data and logic into an `Experiment`.

In [10]:
experiment = algorithm.make_experiment(
	datapipeline_id = datapipeline.id
	, hyperparameters = hyperparameters
	, description = "primarily comparing epochs and lr"
)

In [11]:
batch = experiment.batch

In [12]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:22<00:00,  2.76s/it]


In [13]:
batch.metrics_to_pandas()

,job_id,split,roc_auc,accuracy,precision,recall,f1,loss
0,69,test,0.990741,0.972222,0.974359,0.972222,0.972174,0.144671
1,69,validation,0.995294,0.960000,0.964444,0.960000,0.959843,0.110496
2,69,train,1.000000,1.000000,1.000000,1.000000,1.000000,0.035898
3,70,test,1.000000,1.000000,1.000000,1.000000,1.000000,0.023039
4,70,validation,1.000000,0.960000,0.964444,0.960000,0.959843,0.122498
5,70,train,1.000000,0.988764,0.989126,0.988764,0.988761,0.042143
6,71,test,1.000000,1.000000,1.000000,1.000000,1.000000,0.021097
7,71,validation,1.000000,0.920000,0.936000,0.920000,0.918730,0.132168
8,71,train,1.000000,0.977528,0.978933,0.977528,0.977503,0.053429
9,72,test,1.000000,0.972222,0.974359,0.972222,0.972174,0.065750


### Metrics & Visualization

For more information of visualization of performance metrics, reference the **Visualization & Metrics** documentation.